Code the determine which CI/CD tool has been used.

In [ ]:
import requests
 
# GitHub Personal Access Token - Replace with your GitHub token
GITHUB_TOKEN = ''
# GitHub API URL for the repository contents
#repo_path = 'owner/repo'  # Replace with the target repository path
#api_url = f'https://api.github.com/repos/{repo_path}/contents/'
 
# Common CI/CD configuration paths
ci_cd_files = {
    '.travis.yml': 'Travis CI',
    'Jenkinsfile': 'Jenkins',
    '.gitlab-ci.yml': 'GitLab CI',
    'appveyor.yml': 'AppVeyor',
}


headers = {'Authorization': f'token {GITHUB_TOKEN}'}
 
def find_ci_cd_tools(api_url, headers):
    ci_cd_tools = []
    path_list = []

    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        contents = response.json()
 
        # Check each file or directory in the root of the repository
        for item in contents:
            path_list.append(item.get('path'))

        for path in path_list:
            for ci_cd_file, tool_name in ci_cd_files.items():
                    if path.startswith(ci_cd_file):
                        ci_cd_tools.append(tool_name)

        # for github actions
        if ".github" in path_list:
            actions_api_url=api_url+".github/workflows"
            response = requests.get(actions_api_url, headers=headers)
            response.raise_for_status()
            contents = response.json()
            path = contents[0].get('path')
            if path.startswith(".github/workflows"):
                ci_cd_tools.append("GitHub Actions")

         # for circleci
        if ".circleci" in path_list:
            circle_api_url=api_url+".circleci/config.yml"
            response = requests.get(circle_api_url, headers=headers)
            response.raise_for_status()
            contents = response.json()
            path = contents.get('path')
            if path.startswith(".circleci/config.yml"):
                ci_cd_tools.append("Circle CI")
 
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
 
    return ci_cd_tools


# api_url = f'https://api.github.com/repos/jagrosh/MusicBot/contents/'
# ci_cd_tools_used = find_ci_cd_tools(api_url, headers)
# if ci_cd_tools_used:
#     print("CI/CD tools detected in the repository:", ', '.join(set(ci_cd_tools_used)),"\n")
# else:
#     print("No known CI/CD tools detected in the repository.\n")

with open("./content/java.txt", 'r') as f, open("./content/java_output.txt",'a') as f1:
  for line in f:
    repo_path = str(line)[:-1]
    api_url = f'https://api.github.com/repos/{repo_path}/contents/'
    ci_cd_tools_used = find_ci_cd_tools(api_url, headers)
    if ci_cd_tools_used:
        f1.write(repo_path+", "+', '.join(set(ci_cd_tools_used)) + "\n")
        print(repo_path,":", ', '.join(set(ci_cd_tools_used)))
    else:
        f1.write(repo_path+", "+"No CI/CD found." + "\n")
        print(repo_path,": No known CI/CD tools detected in the repository.")
  f1.close()

Code to find the tests based on the CI/CD tools used.

In [ ]:
import requests

GITHUB_TOKEN = ''
headers = {'Authorization': f'token {GITHUB_TOKEN}'}

unittest_list=["junit", "testng", "spock", "mockito", "cucumber", "dbunit", "nunit", "gauge", "assertions", "mocks", "easymock", "powermock", "boost", "cantata", "cppunit", "gunit", "unittest", "embunit", "boost.test", "ctest", "emma", "mstest", "nunit", "xunit"]
code_smell=["busscout", "checkstyle", "coderush", "codescene", "codesonar", "coverity", "cppcheck", "cppdepend", "findbugs", "flawfinder", "fxcop", "klocwork", "ncrunch", "ndepend", "pmd", "resharper", "resharper", "sonarqube", "sonar", "stylecop"]
security_test=["codeql","snyk", "sqlmap", "appscan", "aquasec", "checkmarx", "sonarqube", "sonar"]

def find_tests(api_url):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        contents = response.json()
        download_url = contents.get("download_url")
        file_content = requests.get(download_url,headers=headers).text
        unit_test_list = ""
        code_smell_list = ""
        security_test_list = ""
        for i in unittest_list:
            if i in file_content:
                unit_test_list = unit_test_list+","+i
        if unit_test_list:
            print(repo_path,":unit test",unit_test_list)
        for i in code_smell:
            if i in file_content:
                code_smell_list = code_smell_list+","+i
        if code_smell_list:
            print(repo_path,":code smell test",code_smell_list)
        for i in security_test:
            if i in file_content:
                security_test_list = security_test_list+","+i
        if security_test_list:
            print(repo_path,":security test",security_test_list)
        return
    else:
        print(f"Failed to fetch content: HTTP {response.status_code}")

with open("./java_output_new","r") as f:
    for line in f:
        repo_path, citool1 = str(line).split(":")
        api_url = f'https://api.github.com/repos/{repo_path.strip()}/contents/'
        ci_tool = citool1.strip().split(",")
        
        for tool in ci_tool:
        # Search in Jenkins
            if tool == "Jenkins":
                jenkins_api_url=api_url+"Jenkinsfile"
                find_tests(jenkins_api_url)

            if tool == "AppVeyor":
                appveyor_api_url = api_url+"appveyor.yml"
                find_tests(appveyor_api_url)

            if tool == "GitLab CI":
                gitlab_api_url = api_url+".gitlab-ci.yml"
                find_tests(gitlab_api_url)

            if tool == "Travis CI":
                travis_api_url = api_url+".travis.yml"
                find_tests(travis_api_url)

            if tool == "Circle CI":
                circleci_api_url = api_url+".circleci/config.yml"
                find_tests(circleci_api_url)

            if tool == "GitHub Actions":
                github_api_url = api_url+".github/workflows"
                response = requests.get(github_api_url, headers=headers)
                if response.status_code == 200:
                    contents = response.json()
                    for yml_files in contents:
                        download_url=yml_files.get("download_url")
                        if download_url is not None and download_url.endswith(".yml"):
                            file_content = requests.get(download_url,headers=headers).text
                            unit_test_list = ""
                            code_smell_list = ""
                            security_test_list = ""
                            for i in unittest_list:
                                if i in file_content:
                                    unit_test_list = unit_test_list+","+i
                            if unit_test_list:
                                print(repo_path,":unit test",unit_test_list)
                            for i in code_smell:
                                if i in file_content:
                                    code_smell_list = code_smell_list+","+i
                            if code_smell_list:
                                print(repo_path,":code smell test",code_smell_list)
                            for i in security_test:
                                if i in file_content:
                                    security_test_list = security_test_list+","+i
                            if security_test_list:
                                print(repo_path,":security test",security_test_list)
                else:
                    print(f"Failed to fetch content: HTTP {response.status_code}")